In [15]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [1]:
import os

In [40]:
os.listdir("data/hw")

['taxi_zone_lookup.csv']

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/06/24 19:17:04 WARN Utils: Your hostname, jose-MacBookPro resolves to a loopback address: 127.0.1.1; using 192.168.1.224 instead (on interface wlp2s0)
23/06/24 19:17:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/24 19:17:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/24 19:17:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
import pandas as pd

In [8]:
df = spark.read.parquet("data/hvfhv/fhvhv_tripdata_2021-06.parquet")

In [9]:
df.count()

14961892

In [10]:
output_path = 'data/hvfhv/2001/06'

df \
    .repartition(12) \
    .write.parquet(output_path)

In [11]:
df_ptn = spark.read.parquet(output_path)

In [44]:
df_ptn.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'originating_base_num',
 'request_datetime',
 'on_scene_datetime',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'base_passenger_fare',
 'tolls',
 'bcf',
 'sales_tax',
 'congestion_surcharge',
 'airport_fee',
 'tips',
 'driver_pay',
 'shared_request_flag',
 'shared_match_flag',
 'access_a_ride_flag',
 'wav_request_flag',
 'wav_match_flag']

In [39]:
df_ptn \
    .select('pickup_datetime', 'dropoff_datetime') \
    .withColumn(
        'duration_hours',
        (F.unix_timestamp(F.col('dropoff_datetime')) - F.unix_timestamp(F.col('pickup_datetime'))) / 3600 ) \
    .select(F.max('duration_hours')).show()

+-------------------+
|max(duration_hours)|
+-------------------+
|   66.8788888888889|
+-------------------+



In [42]:
zd = spark.read \
    .option("header", "true") \
    .csv("data/hw/taxi_zone_lookup.csv")

In [47]:
zd.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [ ]:
df_ptn.join(zd, df_ptn.PULocationID == zd.LocationID).groupBy("Zone").agg(F.count("Zone").alias("zcount")).orderBy(F.desc(F.col('zcount'))).show()

In [9]:
year = 2021 

for month in range(1, 9):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1


AnalysisException: path file:/home/jose/Documents/projects/de-zoomcamp-fa/wk5/data/pq/green/2021/01 already exists.

In [10]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [11]:
year = 2021

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7


processing data for 2021/8
processing data for 2021/9


AnalysisException: Path does not exist: file:/home/jose/Documents/projects/de-zoomcamp-fa/wk5/data/raw/yellow/2021/09